In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.optimize import curve_fit
import re

In [4]:
Tokyo_npy_divide_PATH = '/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Tokyostation_2021/Tokyostation_swarm_3zi_2021.npy'

In [16]:
df = pd.read_csv('/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Tokyostation_2021/processed_Tokyostation_3zi_2021.csv')

In [30]:
columns = df.columns

In [31]:
columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'text', 'created_at',
       'retweets', 'replies', 'likes', 'quote_count', 'place_id', 'author_id',
       'username', 'author_followers', 'author_tweets', 'author_description',
       'author_location', 'creatid_at_month', 'creatid_at_day',
       'creatid_at_hour'],
      dtype='object')

In [33]:
df = df[(df['text'].str.contains('I\'m at')) | (df['text'].str.contains('@') & df['text'].str.contains('in'))]

In [34]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,place_id,author_id,username,author_followers,author_tweets,author_description,author_location,creatid_at_month,creatid_at_day,creatid_at_hour
9,9,9,9,上野東京ライン最終高崎行き乗車。最終が東京始発は有難い。最終っていつぶりだろう？ (@ 7-...,2021-12-31 23:16:53,0.0,0.0,0.0,0.0,28b9063fdce43645,1.362795e+08,pp411,289.0,66006.0,相互フォローは期待しないで下さい。ハロヲタ。殿堂入りって何？ 推し：安倍なつみ・真野恵里菜・...,群馬県,12,31,23
10,10,10,10,I'm at 新幹線北のりかえ口 in Chiyoda Tōkyō https://t.co...,2021-12-31 23:08:18,0.0,0.0,2.0,0.0,28b9063fdce43645,1.240236e+18,Unlock_Frozen_,761.0,154684.0,競馬/村上まなつ/和氣あず未/DIALOGUE+ アイコンはしる粉さん(@s_r_k0730)より,紅芋タルト,12,31,23
11,11,11,11,I'm at 東京駅 in 千代田区 東京都 https://t.co/2riHaJGfvl,2021-12-31 23:08:13,0.0,0.0,0.0,0.0,28b9063fdce43645,1.240236e+18,Unlock_Frozen_,761.0,154684.0,競馬/村上まなつ/和氣あず未/DIALOGUE+ アイコンはしる粉さん(@s_r_k0730)より,紅芋タルト,12,31,23
13,13,13,13,This year's NHK #Kouhaku makes me miss those d...,2021-12-31 22:58:57,0.0,0.0,1.0,0.0,28b9063fdce43645,4.334442e+07,1984ews,625.0,13512.0,幕后黑熊 the Kuma Behind the Stage,Valhalla Crater Callisto,12,31,22
15,15,15,15,I'm at 東京駅 in 千代田区 東京都 https://t.co/r2O68aaUPK,2021-12-31 22:44:17,0.0,0.0,0.0,0.0,28b9063fdce43645,1.281920e+08,calsonic1994,628.0,469807.0,本名・素顔フリー素材協会代表。 東京の平和なとこ住んでます。美味しいものときゃわわを求めて各...,美人ときゃわわの生息する場所,12,31,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87565,89153,5623,5506,I'm at 東北新幹線 東京駅 in 千代田区 東京都 https://t.co/S4Lq...,2021-01-01 06:02:22,0.0,0.0,0.0,0.0,28b9063fdce43645,1.443730e+09,shizuna_magica,632.0,194956.0,静岡県出身。乗り物は絶対に窓側派。旅行、走行音の全区間録音、鉄道の動画撮影、大相撲観戦が趣味...,神奈川の限りなく静岡寄り,1,1,6
87566,89154,5624,5507,I'm at JR 東京駅 - @jreast_official in 千代田区 東京都 h...,2021-01-01 06:02:05,0.0,0.0,0.0,0.0,28b9063fdce43645,1.443730e+09,shizuna_magica,632.0,194956.0,静岡県出身。乗り物は絶対に窓側派。旅行、走行音の全区間録音、鉄道の動画撮影、大相撲観戦が趣味...,神奈川の限りなく静岡寄り,1,1,6
87567,89155,5625,5508,I'm at 東京駅 in 千代田区 東京都 https://t.co/9SAQIoR4BF,2021-01-01 06:01:49,0.0,0.0,0.0,0.0,28b9063fdce43645,1.443730e+09,shizuna_magica,632.0,194956.0,静岡県出身。乗り物は絶対に窓側派。旅行、走行音の全区間録音、鉄道の動画撮影、大相撲観戦が趣味...,神奈川の限りなく静岡寄り,1,1,6
87568,89156,5626,5509,I'm at 東京駅 in 千代田区 東京都 https://t.co/zEwh0ccUcp,2021-01-01 05:42:28,0.0,0.0,0.0,0.0,28b9063fdce43645,5.809131e+08,Hou1323,752.0,147401.0,座右の銘は「自分をたくさん甘やかしてあげてね」です,NaN,1,1,5


In [39]:
day_list_long = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
day_list_short = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
day_list_Feb = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
hour_list = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

In [42]:
list_num_day = []
estimate_from_mobile_data = []
for month in [1,2,3,4,5,6,7,8,9,10,11,12]:
    df_onemonth =  df[df['creatid_at_month']==month]
    if month in [1, 3, 5, 7, 8,10, 12]:
        day_list = day_list_long
    elif month == 2:
        day_list = day_list_Feb
    else:
        day_list = day_list_short
    for day in day_list:
        list_num_hour = []
        for hour in hour_list:
            df_oneday = df_onemonth[df_onemonth['creatid_at_day']==day]
            num = len(df_oneday[df_oneday['creatid_at_hour']==hour])
            list_num_hour.append(num)
        list_num_day.append(list_num_hour)

In [43]:
list_num_day = np.array(list_num_day)

In [44]:
list_num_day

array([[ 0,  0,  0, ...,  5,  1,  6],
       [ 1,  0,  0, ...,  3,  5,  0],
       [ 0,  0,  0, ...,  7,  6,  4],
       ...,
       [ 0,  0,  0, ..., 22,  8, 19],
       [ 2,  0,  0, ..., 21,  4, 10],
       [ 1,  0,  0, ...,  9,  8,  3]])

In [45]:
np.save(Tokyo_npy_divide_PATH, list_num_day)